# Ant Colony

Ant Colony optimization (ACO) is an algorithm used to solve combinatorial problem, such as the travelling salesman problem: suppose we must vist some cities and can do it in any order, what is the best way to travel between the cities so we minimize journey time?

The "ants" starts at random cities and leave "pheremone" trails, the more ants go a partiular route the more strong the pheremones, and the stronger the pheremones the more ants will travel that way. Eventually, we are left with a single trail of pheremones which represents the optimal solution found by ACO. 

In [1]:
import random as r
import numpy as np
r.seed(1)

## Objective Functions

In this notebook we'll test the semi-empirical mass formula. It's a formula from physics which gives us the binding energy of a nucleus given its proton number, Z, and neutron number, N. Since physics is effectively trying to maximise the binding energy per nucleon, we have a fun optimization problem if we divide the energy predicted by the SEMF by Z + N. You can play around with physics and see which isotopes would be stable if you changed, say, the Coulomb term! 

SEMF is also useful since it has a known best result of Nickel-62: Z = 28, N = 34. The closer the output is to this, the better our algorithm is doing. Another very stable isotope is Iron-56: Z = 26, N = 30, so if we get a result close to this the algorithm has done well, too.

More about the SEMF here: https://en.wikipedia.org/wiki/Semi-empirical_mass_formula

If you want a different objective function, check out Objectives.ipynb.

In [2]:
nCities = 25

def generateCity():
    return(r.uniform(0, 100), r.uniform(0, 100))

def calculateDistance(city1, city2):
    output = 0
    for x, y in city1, city2:
        output += (x - y) ** 2
    output = output ** 0.5
    return(output)

citySet = []
for i in range(nCities):
    citySet.append(generateCity())

def loss(cityOrdering):
    output = 0
    for i in range(len(citySet) - 1):
        output += calculateDistance(citySet[cityOrdering[i]], citySet[cityOrdering[i + 1]]) 
    return(output)

print(loss([x for x in range(nCities)]))

1208.0131349320782


## Ant Colony

In [30]:
class Ant:
    def __init__(self,
                citySet,
                colony):
        self.cities = citySet
        self.location = r.randint(0, len(self.cities) - 1)
        self.destination = r.randint(0, len(self.cities) - 1)
        while self.destination == self.location:
            self.destination = r.randint(0, len(self.cities) - 1)
        self.distanceToDestination = calculateDistance(citySet[self.location], citySet[self.destination])
        self.colony = colony

    def updateAntOneEpisode(self, antSpeed = 1):
        self.distanceToDestination -= antSpeed
        if self.distanceToDestination < 0:
            self.sendPheremones(self.location, self.destination)
            self.location = self.selectNewDestination()
            while self.destination == self.location:
                self.destination = r.randint(0, len(self.cities) - 1)
            self.distanceToDestination = calculateDistance(citySet[self.location], citySet[self.destination])

    #Increase the pheremones from x to y
    def sendPheremones(self, x, y):
        self.colony.pheremoneMatrix[x][y] += 100
        magnitude = np.sum(self.colony.pheremoneMatrix[x])
        self.colony.pheremoneMatrix[x] = [(c / magnitude) for c in self.colony.pheremoneMatrix[x]]
        return(self.colony.pheremoneMatrix)

    def selectNewDestination(self):
        prob = r.uniform(0, 1)
        i = -1
        while prob > 0:
            i += 1
            prob -= self.colony.pheremoneMatrix[self.location][i]
        return(i)
            

class AntColony:
    def __init__(
        self,
        cities
    ):
        self.cities = cities
        self.antLocations = [Ant(cities, self) for i in range(len(self.cities) - 1)]
        self.pheremoneMatrix = self.generatePheremoneMatrix()

    def generatePheremoneMatrix(self):
        output = []
        for i in range(len(self.cities)):
            row = []
            for j in range(len(self.cities)):
                row.append((1/len(self.cities)))
            output.append(row)
        return(output)
        
    def optimize(self, numIterations = 25000):
        for i in range(numIterations):
            for ant in self.antLocations:
                ant.updateAntOneEpisode()
        return(self.pheremoneMatrix)

    def interpretPheremones(self):
        output = []
        for x in self.pheremoneMatrix:
            row = []
            for i in x:
                if i > 0.9:
                    row.append(1)
                else:
                    row.append(0)
            output.append(row)
        return(output)
ac = AntColony(citySet)
ac.optimize()
for x in ac.interpretPheremones():
    assert(np.sum(x) == 1) #Check solution is valid
    print(x)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0,